In [114]:
import torch
import os
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset , DataLoader
import zipfile
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import csv

In [115]:
#with zipfile.ZipFile("archive.zip" ,"r") as zip_ref:
    #zip_ref.extractall("dataset")

In [116]:
image_train = r"C:\Users\ASUS\OneDrive\Masaüstü\traffic-sign-classifier\dataset\Train"

for number , class_train in enumerate(os.listdir(image_train)):
    print(number , class_train)
    

0 0
1 1
2 10
3 11
4 12
5 13
6 14
7 15
8 16
9 17
10 18
11 19
12 2
13 20
14 21
15 22
16 23
17 24
18 25
19 26
20 27
21 28
22 29
23 3
24 30
25 31
26 32
27 33
28 34
29 35
30 36
31 37
32 38
33 39
34 4
35 40
36 41
37 42
38 5
39 6
40 7
41 8
42 9


In [117]:
csv_path = r"C:\Users\ASUS\OneDrive\Masaüstü\traffic-sign-classifier\dataset\signnames.csv"

with open(csv_path, newline='', encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        print(row["ClassId"], "→", row["SignName"])


0 → Speed limit (20km/h)
1 → Speed limit (30km/h)
2 → Speed limit (50km/h)
3 → Speed limit (60km/h)
4 → Speed limit (70km/h)
5 → Speed limit (80km/h)
6 → End of speed limit (80km/h)
7 → Speed limit (100km/h)
8 → Speed limit (120km/h)
9 → No passing
10 → No passing for vehicles over 3.5 metric tons
11 → Right-of-way at the next intersection
12 → Priority road
13 → Yield
14 → Stop
15 → No vehicles
16 → Vehicles over 3.5 metric tons prohibited
17 → No entry
18 → General caution
19 → Dangerous curve to the left
20 → Dangerous curve to the right
21 → Double curve
22 → Bumpy road
23 → Slippery road
24 → Road narrows on the right
25 → Road work
26 → Traffic signals
27 → Pedestrians
28 → Children crossing
29 → Bicycles crossing
30 → Beware of ice/snow
31 → Wild animals crossing
32 → End of all speed and passing limits
33 → Turn right ahead
34 → Turn left ahead
35 → Ahead only
36 → Go straight or right
37 → Go straight or left
38 → Keep right
39 → Keep left
40 → Roundabout mandatory
41 → End of

In [118]:
class ImageDataset(Dataset):
    def __init__(self ,image_train , transform):
        self.image_train = image_train
        self.image_paths = []
        self.labels = []
        self.class_name = {}
        self.transform = transform
        self.id_to_name = None

        csv_path = r"C:\Users\ASUS\OneDrive\Masaüstü\traffic-sign-classifier\dataset\signnames.csv"
        names_df = pd.read_csv(csv_path)
        self.id_to_name = dict(zip(names_df["ClassId"], names_df["SignName"]))
            
        for class_id in sorted(os.listdir(image_train), key=lambda x: int(x)):
            class_id = int(class_id)
            self.class_name[class_id] = self.id_to_name[class_id]
            class_path = os.path.join(image_train, str(class_id))
        
            for img_name in os.listdir(class_path):
                self.image_paths.append(os.path.join(class_path , img_name))
                self.labels.append(class_id)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        

        if self.transform:
            image = self.transform(image)

        return image , label 

In [119]:
class TestImageDataset(Dataset):
    def __init__(self, test_csv_path, base_folder, transform=None):
        self.transform = transform
        self.base_folder = base_folder
        self.image_paths = []
        self.labels = []
        test_df = pd.read_csv(test_csv_path)
        for index, row in test_df.iterrows():
            image_name = row['Path'].split('/')[-1]
            image_path = os.path.join(self.base_folder, image_name)
            
            self.image_paths.append(image_path)
            self.labels.append(row['ClassId'])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        

        if self.transform:
            image = self.transform(image)

        return image, label

In [120]:
transform = transforms.Compose([

    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [-0.3194, -0.3756, -0.3571] , std = [0.3261, 0.3260, 0.3454])
])

In [121]:
image_train = r"C:\Users\ASUS\OneDrive\Masaüstü\traffic-sign-classifier\dataset\Train"
image_test = r"C:\Users\ASUS\OneDrive\Masaüstü\traffic-sign-classifier\dataset\Test"

image_train_dataset = ImageDataset (image_train , transform = transform)
image_test_dataset = TestImageDataset(r"C:\Users\ASUS\OneDrive\Masaüstü\traffic-sign-classifier\dataset\Test.csv",image_test , transform = transform)


In [122]:
train_image_loader = DataLoader(dataset= image_train_dataset , batch_size = 32 , shuffle = True, num_workers=0)
test_image_loader = DataLoader(dataset= image_test_dataset , batch_size = 32 , shuffle = False, num_workers=0)

In [123]:
for images, labels in train_image_loader:
    print(images.shape, labels.shape)

    img = images[0].numpy()
    print(img.shape)

    label = labels[0].item()
    print(image_train_dataset.class_name[label])

    img = np.transpose(img, (1, 2, 0))
    print(img.shape)
    print(label)
    break


torch.Size([32, 3, 64, 64]) torch.Size([32])
(3, 64, 64)
Priority road
(64, 64, 3)
12


Custom CNN Architecture

In [124]:
import torch.nn as nn
import torch.optim as optim


In [125]:
class CustomCnnModel(nn.Module):
    def __init__(self , input_dim, num_classes):
        super(CustomCnnModel,self).__init__()
        self.input_dim = input_dim
        self.num_classes = num_classes

        self.conv_layers = nn.Sequential(

            #FC1
            nn.Conv2d(3 ,32 , kernel_size=3 , stride=1 , padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2 ,stride=2),

            #FC2
            nn.Conv2d(32 ,64 , kernel_size=3 , stride=1 , padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2 ,stride=2),

            #FC3
            nn.Conv2d(64 ,128 , kernel_size=3 , stride=1 , padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2 ,stride=2),

            #FC4
            nn.Conv2d(128 ,256 , kernel_size=3 , stride=1 , padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2 ,stride=2)

        )

        self._to_linear = None
        self._get_conv_output(self.input_dim)


        self.fc_layers = nn.Sequential(
            nn.Linear(self._to_linear ,512),
            nn.ReLU(),
            nn.Linear(512,128),
            nn.ReLU(),
            nn.Linear(128,self.num_classes),


        )
    def _get_conv_output(self, input_dim=64):
        with torch.no_grad():
            dummy_input = torch.zeros(1,3 ,input_dim , input_dim)
            output = self.conv_layers(dummy_input)
            self._to_linear  = output.view(1 , -1).size(1)
           
    
    
    
    def forward(self,x):
        x = self.conv_layers(x)
        x = x.view(x.size(0) , -1)
        x = self.fc_layers(x)
        return x

In [126]:
#Initialize Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model =CustomCnnModel(input_dim=64 , num_classes=43).to(device)


In [127]:
print(model)

CustomCnnModel(
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [128]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters() , lr=0.001)

In [129]:
# Training loop

epochs = 20
for epoch in range (epochs):
    model.train()
    running_loss = 0.0
    for images , labels in train_image_loader:
        images , labels = images.to(device) , labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs , labels)
        loss.backward()
        optimizer.step()
    running_loss+= loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss :{running_loss/len(train_image_loader)}")

Epoch 1/20, Loss :7.657988717077607e-05
Epoch 2/20, Loss :1.61740330289472e-05
Epoch 3/20, Loss :2.226983088784248e-07
Epoch 4/20, Loss :8.632018665561292e-09
Epoch 5/20, Loss :5.318385170587319e-06
Epoch 6/20, Loss :4.969666735772393e-09
Epoch 7/20, Loss :5.293855417472235e-10
Epoch 8/20, Loss :2.4883915194679435e-05
Epoch 9/20, Loss :6.57929728388958e-09
Epoch 10/20, Loss :1.6826067412679264e-07
Epoch 11/20, Loss :9.07516031509222e-10
Epoch 12/20, Loss :6.633395898185494e-09
Epoch 13/20, Loss :1.9573459179910905e-05
Epoch 14/20, Loss :3.861472486149137e-07
Epoch 15/20, Loss :8.426306097537315e-08
Epoch 16/20, Loss :9.145138662370746e-08
Epoch 17/20, Loss :1.9458836236236342e-06
Epoch 18/20, Loss :4.029759599420441e-09
Epoch 19/20, Loss :1.5880646928347235e-08
Epoch 20/20, Loss :2.7348976779795976e-06


In [130]:
#Evaluate

model.eval()
correct = 0 
total = 0

with torch.no_grad():
    for images , labels in test_image_loader:
        images , labels = images.to(device) , labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        _ , predicted = torch.max(outputs , 1)
        total+= labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy is : {100*correct / total : .2f}%")

Test Accuracy is :  96.22%


In [149]:
import cv2
class ImageClassifier:
    def __init__(self, model_path, class_names):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.class_names = class_names
        self.model = CustomCnnModel(input_dim=64, num_classes=len(class_names)).to(self.device)
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        self.model.eval()
        self.transform = transforms.Compose([
            transforms.Resize((64, 64)),
            transforms.ToTensor(),
            transforms.Normalize(mean = [-0.3194, -0.3756, -0.3571] , std = [0.3261, 0.3260, 0.3454]),
        ])

    def predict(self, image_path, save_annotated="output_image.jpg"):
        pil_img = Image.open(image_path).convert("RGB")
        x = self.transform(pil_img).unsqueeze(0).to(self.device)

        with torch.no_grad():
            logits = self.model(x)
            pred_idx = logits.argmax(1).item()

        label = self.class_names[pred_idx]

        img = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
        cv2.putText(img, label, (0, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        cv2.imwrite(save_annotated, img)
        return label

In [150]:
torch.save(model.state_dict() , "traffic_sign_model.pth")

In [159]:
classifier = ImageClassifier(r"C:\Users\ASUS\OneDrive\Masaüstü\traffic-sign-classifier\traffic_sign_model.pth" , image_train_dataset.class_name)
label = classifier.predict(r"C:\Users\ASUS\OneDrive\Masaüstü\traffic-sign-classifier\dataset\Test\06875.png")
print(f"Predicted class is : {label}")

Predicted class is : Keep right
